<a href="https://colab.research.google.com/github/phoumithona/matrix_computation/blob/master/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm 
import time

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

# normalize the values of the input data to be [0, 1]
def normalize(data):
    data_normalized = (data - min(data)) / (max(data) - min(data))
    return(data_normalized)

# make a matrix each column of which represents an images in a vector form
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:
    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

In [3]:
# split dataset
train_label = list_label[:6000,]  # 6000 rows training label
test_label  = list_label[6000:,]  # 4000 rows testing label
train_image = list_image[:,:6000] # 6000 rows training image
test_image  = list_image[:,6000:] # 4000 rows testing image

# data transposed
x_train = train_image.T
y_train = train_label.T
x_test  = test_image.T
y_test  = test_label.T

print('train_image', x_train.shape)
print('train_label', y_train.shape)
print('test_image', x_test.shape)
print('test_label', y_test.shape)

train_image (6000, 784)
train_label (6000,)
test_image (4000, 784)
test_label (4000,)
